In [1]:
import numpy as np
import scipy as sp
import tensorflow as tf
import tensorflow.contrib.distributions as ds

In [2]:
sess = tf.InteractiveSession()

In [3]:
class GMM(object):
    def __init__(self, mu, sigma, weights, dim):
        # Required parameters 
        self.mu = mu
        self.sigma = sigma
        self.weights = weights
        self.dim = dim
        
    def log_px(self, x):
        # log_px = log(sum(exp(log(w_i) + log(p_i(x)))))
        log_px = []
        for i in range(weights.shape[0]):
            mu_, sigma_ = self.mu[i] * tf.ones(dim), self.sigma[i] * tf.ones(dim)
            mvn = ds.MultivariateNormalDiag(loc=mu_, scale_diag=sigma_)
            # Calculate log_px for each component
            log_px_i = tf.reduce_logsumexp(mvn.log_prob(x)) + tf.log(tf.to_float(weights[i]))
            log_px.append(log_px_i)
        return tf.reduce_logsumexp(log_px)
    
    def d_log_px(self, x):
        # d_log_px = 1 / exp(log(sum(exp(log(w_i) + log(p_i(x)))))) 
        #            * sum(exp(log(w_i) + log(p_i(x)) + log(-(x - mu)/sigma^2)))
        # Use symbolic differentiation instead
        log_px = self.log_px(x)
        return tf.gradients(log_px, [x])

In [4]:
mu = np.array([1., -1.]); sigma = np.sqrt(np.array([0.1, 0.05])); weights = np.array([1./3, 2./3]); dim=6
gmm = GMM(mu, sigma, weights, dim)

In [5]:
def median(x):
    x = tf.reshape(x, [-1])
    med = tf.floordiv(tf.shape(x)[0], 2)
    check_parity = tf.equal(tf.to_float(med), tf.divide(tf.to_float(tf.shape(x)[0]), 2.))
    def is_true():
        return tf.reduce_sum(tf.nn.top_k(x, med+1).values[-2:]) / 2.
    def is_false():
        return tf.nn.top_k(x, med+1).values[-1]
    return tf.cond(check_parity, is_true, is_false) 

In [17]:
class stein_is(object):
    def __init__(self, gmm_model, mu, sigma, dim, n_leaders, n_followers, step_size=0.01): # n_trials, step_size=0.01):
        # Required parameters
        self.gmm_model = gmm_model
        self.mu = mu
        self.sigma = sigma
        self.dim = dim
        self.n_leaders = n_leaders
        self.n_followers = n_followers
        # self.n_trials = n_trials
        self.step_size = step_size
        self.eps = 1e-10
        
        # Set seed
        seed = 30
        
        # Intialisation
        self.B, self.B_density, self.A = self.initialise_variables()
        self.pB = self.gmm_model.log_px(self.B)
        
        # Register functions for debugging
        # self.k_A_A, self.sum_grad_A_k_A_A, self.A_Squared, self.h_0 = self.construct_map()
        # self.k_A_B, self.sum_grad_A_k_A_B, self.grad_A_grad_B_k_A_B, self.grad_B_k_A_B = self.apply_map()        
        self.A, self.B, self.phi_B, self.grad_B_phi_B = self.svgd_update()
        self.q_density = self.density_update()
        
        
    def initialise_variables(self):
        init_distribution = tf.contrib.distributions.MultivariateNormalDiag(self.mu * tf.ones(dim), self.sigma * tf.ones(dim))
        
        # followers = tf.reshape(init_distribution.sample(self.n_trials * self.n_followers, seed=123), [self.n_trials, self.n_followers, self.h_dim] 
        # leaders = tf.reshape(init_distribution.sample(self.n_trials * self.n_leaders, seed=123), [self.n_trials, self.n_leaders, self.h_dim] 
        
        followers = tf.reshape(init_distribution.sample(self.n_followers, seed=123), [self.n_followers, self.dim]) 
        q_density = init_distribution.log_prob(followers)
        leaders = tf.reshape(init_distribution.sample(self.n_leaders, seed=123), [self.n_leaders, self.dim])
                           
        return followers, q_density, leaders
                             
    def construct_map(self):
        # Calculate ||leader - leader'||^2/h_0, refer to leader as A as in SteinIS
        x2_A_A_T = tf.multiply(2., tf.matmul(self.A, tf.transpose(self.A)))
        A_Squared = tf.reduce_sum(tf.square(self.A), 1)
        A_Distance = tf.add(tf.subtract(A_Squared, x2_A_A_T), tf.transpose(A_Squared))   
        # h_0 = tf.divide(tf.add(median(A_Distance), self.eps), 2. * (tf.log(tf.cast(self.n_leaders, tf.float32)) + 1.))
        h_0 = tf.divide(median(A_Distance), 2. * (tf.log(tf.to_float(self.n_leaders)) + 1.))
        k_A_A = tf.exp(-tf.div(A_Distance, tf.square(h_0)))
        sum_grad_A_k_A_A = tf.reduce_sum(tf.gradients(k_A_A, [self.A]), 1)
        return k_A_A, sum_grad_A_k_A_A, A_Squared, h_0
    
    def apply_map(self):
        # Calculate ||leader - follower||^2/h_0, refer to follower as B as in SteinIS
        x2_A_B_T = tf.multiply(2., tf.matmul(self.A, tf.transpose(self.B)))
        B_Squared = tf.reduce_sum(tf.square(self.B), 1)
        A_B_Distance  = tf.add(tf.subtract(self.A_Squared, x2_A_B_T), B_Squared)
        k_A_B = tf.exp(-tf.div(A_B_Distance, tf.square(self.h_0)))
        sum_grad_A_k_A_B = tf.reduce_sum(tf.gradients(k_A_B, [self.A]), 1)
        return k_A_B, sum_grad_A_k_A_B 
                    
    def svgd_update(self):
        self.k_A_A, self.sum_grad_A_k_A_A, self.A_Squared, self.h_0 = self.construct_map()
        self.k_A_B, self.sum_grad_A_k_A_B = self.apply_map()
        self.d_log_pA = self.gmm_model.d_log_px(self.A)[0]
        sum_d_log_pA_T_k_A_A = tf.reduce_sum(tf.matmul(self.k_A_A, self.d_log_pA), 0)       
        phi_A = (1. / tf.to_float(self.n_leaders)) * tf.add(sum_d_log_pA_T_k_A_A, self.sum_grad_A_k_A_A)
        A = tf.add(self.A, self.step_size * phi_A)  
        sum_d_log_pA_T_k_A_B = tf.reduce_sum(tf.matmul(self.k_A_B, self.d_log_pA), 0)       
        phi_B = (1. / tf.to_float(self.n_leaders)) * tf.add(sum_d_log_pA_T_k_A_B, self.sum_grad_A_k_A_B)
        B = tf.add(self.B, self.step_size * phi_B) 
        grad_B_phi_B = tf.gradients(phi_B, [self.B])
        return A, B, phi_B, grad_B_phi_B[0]
    
    def density_update(self):
        I = tf.eye(self.dim)
        inv_abs_det_I_grad_B_phi = tf.map_fn(lambda x: 1./tf.abs(tf.matrix_determinant(tf.add(I, x))), self.grad_B_phi_B)
        return tf.multiply(self.B_density, inv_abs_det_I_grad_B_phi) 
    
    def main(self, iteration):
        for i in range(iteration):
            self.A, self.B, self.phi_B, self.grad_B_phi_B = self.svgd_update()
            self.q_density = self.density_update()
            print 'Iteration ', str(i), ' done'
        self.importance_weights = tf.divide(self.q_density, self.pB)
        self.normalisation_constant = 1./tf.to_float(self.n_followers) * tf.reduce_sum(self.importance_weights)
        self.final_B = self.B
        return self.normalisation_constant
#         return self.final_B, self.importance_weights, self.normalisation_constant
#         return self.A, self.B

In [18]:
mu = 0.; sigma = 3.; dim = 6; n_leaders = 100; n_followers = 100;
model = stein_is(gmm,  mu, sigma, dim, n_leaders, n_followers)

In [19]:
sess.run([model.main(100)])

Iteration  0  done
Iteration  1  done
Iteration  2  done
Iteration  3  done
Iteration  4  done
Iteration  5  done
Iteration  6  done
Iteration  7  done
Iteration  8  done
Iteration  9  done
Iteration  10  done
Iteration  11  done
Iteration  12  done
Iteration  13  done
Iteration  14  done
Iteration  15  done
Iteration  16  done
Iteration  17  done
Iteration  18  done
Iteration  19  done
Iteration  20  done
Iteration  21  done
Iteration  22  done
Iteration  23  done
Iteration  24  done
Iteration  25  done
Iteration  26  done
Iteration  27  done
Iteration  28  done
Iteration  29  done
Iteration  30  done
Iteration  31  done
Iteration  32  done
Iteration  33  done
Iteration  34  done
Iteration  35  done
Iteration  36  done
Iteration  37  done
Iteration  38  done
Iteration  39  done
Iteration  40  done
Iteration  41  done
Iteration  42  done
Iteration  43  done
Iteration  44  done
Iteration  45  done
Iteration  46  done
Iteration  47  done
Iteration  48  done
Iteration  49  done
Iteration 

[0.82154536]